In [6]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
sys.path.append('../LIB/')
from env import ENV
from sklearn.preprocessing import normalize
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import coo_matrix, hstack
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold
from itertools import combinations

In [7]:
train = pd.read_pickle(ENV.lgb_train_0827_raw.value)
print('train shape is: {}'.format(train.shape))
test = pd.read_pickle(ENV.lgb_test_0827_raw.value)
print('test shape is: {}'.format(test.shape))
target = train.TARGET.values



train shape is: (307511, 409)
test shape is: (48744, 408)


In [8]:
train_ori = pd.read_csv(ENV.application_train_ori.value)
test_ori = pd.read_csv(ENV.application_test_ori.value)

train_m = train_ori[['SK_ID_CURR','AMT_CREDIT','AMT_INCOME_TOTAL','AMT_GOODS_PRICE']].copy()
test_m = test_ori[['SK_ID_CURR','AMT_CREDIT','AMT_INCOME_TOTAL','AMT_GOODS_PRICE']].copy()

train = train.merge(train_m,how='left',left_on='SK_ID_CURR',right_on='SK_ID_CURR')
test = test.merge(test_m,how='left',left_on='SK_ID_CURR',right_on='SK_ID_CURR')

X = pd.concat([train.drop('TARGET',axis=1),test])

In [12]:
income = 'AMT_INCOME_TOTAL'
goods = 'AMT_GOODS_PRICE'
tr = 200
for col in tqdm(X.columns):
    if X[col].nunique() > tr:
        if col.lower().find('AMT_INCOME_TOTAL') == -1 and col != 'SK_ID_CURR':
            print(col)
            col_name = 'Wei_' + col +'==DEVIDE==AMT_INCOME_TOTAL'
            X[col_name] = X[col] / X[income]
        elif col.lower().find('AMT_GOODS_PRICE') == -1 and col != 'SK_ID_CURR':
            col_name = 'Wei_' + col +'==DEVIDE==AMT_GOODS_PRICE'
            X[col_name] = X[col] / X[goods]

  5%|▍         | 36/790 [00:00<00:04, 173.47it/s]

AMT_CREDIT_divide_AMT_ANNUITY
external_sources_min
AMT_CREDIT_divide_AMT_GOODS_PRICE
external_sources_max
DAYS_BIRTH_x
bureau_AMT_CREDIT_SUM_divide_AMT_CREDIT_SUM_DEBT_min
AMT_ANNUITY_divide_DAYS_EMPLOYED
EXT_SOURCE_2_x
EXT_SOURCE_3_x
NAME_EDUCATION_TYPE_CODE_GENDER_EXT_SOURCE_3_mean_abs_diff
OCCUPATION_TYPE_DAYS_BIRTH_mean_abs_diff
EXT_SOURCE_1_x
NAME_FAMILY_STATUS_NAME_EDUCATION_TYPE_AMT_ANNUITY_max_diff
NAME_EDUCATION_TYPE_CODE_GENDER_AMT_ANNUITY_max_diff
bureau_debt_credit_ratio
NAME_EDUCATION_TYPE_CODE_GENDER_DAYS_EMPLOYED_mean_abs_diff
AMT_ANNUITY_divide_DAYS_LAST_PHONE_CHANGE
bureau_agg__AMT_CREDIT_SUM_divide_DAYS_CREDIT_max_minus_min
NAME_EDUCATION_TYPE_CODE_GENDER_REGION_POPULATION_RELATIVE_mean_abs_diff
bureau_AMT_CREDIT_SUM_divide_DAYS_CREDIT_ENDDATE_std
ACTIVE_DAYS_CREDIT_MAX
ACTIVE_DAYS_CREDIT_ENDDATE_MIN
AMT_ANNUITY_x
AMT_INCOME_TOTAL_divide_DAYS_LAST_PHONE_CHANGE
DAYS_EMPLOYED_x
CNT_INSTALMENT_FUTURE_mean
DAYS_ID_PUBLISH_x
last_loan_installment_paid_late_in_days_std
AMT_

  9%|▉         | 71/790 [00:00<00:04, 171.69it/s]

prev_AMT_ANNUITY_divide_CNT_PAYMENT_median
NAME_FAMILY_STATUS_CODE_GENDER_AMT_INCOME_TOTAL_max_diff
bureau_DAYS_CREDIT_ENDDATE_max
previous_application_fraction_of_refused_applications
prev_HOUR_APPR_PROCESS_START_mean
prev_DAYS_LAST_DUE_1ST_VERSION_minus_DAYS_LAST_DUE_min
AMT_INCOME_TOTAL_divide_AMT_ANNUITY
NAME_EDUCATION_TYPE_CODE_GENDER_EXT_SOURCE_2_mean_abs_diff
last_10by50_fraction_installment_paid_late_in_days_kurt
ACTIVE_DAYS_CREDIT_ENDDATE_MEAN
bureau_AMT_CREDIT_SUM_divide_DAYS_CREDIT_min
prev_AMT_ANNUITY_divide_DAYS_LAST_DUE_1ST_VERSION_min
NAME_EDUCATION_TYPE_CODE_GENDER_DAYS_BIRTH_mean_abs_diff
12_period_trend_CNT_INSTALMENT_FUTURE
annuity_income_percentage
CLOSED_DAYS_CREDIT_MAX
10_period_trend_installment_paid_late_in_days
credit_to_income_ratio
inst_AMT_CUM_PAYMENT_sum
NAME_EDUCATION_TYPE_CODE_GENDER_DAYS_ID_PUBLISH_mean_abs_diff
bureau_agg__DAYS_CREDIT_normed_std
NAME_FAMILY_STATUS_NAME_EDUCATION_TYPE_DAYS_BIRTH_mean_abs_diff
YEARS_BEGINEXPLUATATION_AVG_x
last_5by20_frac

 14%|█▍        | 109/790 [00:00<00:03, 176.75it/s]

...

Wei_inst_slope_AMT_LATE_PAYMENT_max_mean==DEVIDE==AMT_INCOME_TOTAL
Wei_ACTIVE_AMT_CREDIT_SUM_LIMIT_MEAN==DEVIDE==AMT_INCOME_TOTAL
Wei_prev_AMT_APPLICATION_divide_AMT_GOODS_PRICE_mean==DEVIDE==AMT_INCOME_TOTAL
Wei_NAME_EDUCATION_TYPE_CODE_GENDER_OWN_CAR_AGE_mean_abs_diff==DEVIDE==AMT_INCOME_TOTAL
Wei_prev_AMT_ANNUITY_divide_DAYS_FIRST_DUE_max==DEVIDE==AMT_INCOME_TOTAL
Wei_bureau_AMT_CREDIT_MAX_OVERDUE_divide_AMT_CREDIT_SUM_max==DEVIDE==AMT_INCOME_TOTAL
Wei_inst_LATE_DAYS_SIGN_area_RATE_LATE_PAYMENT_mean==DEVIDE==AMT_INCOME_TOTAL
Wei_inst_AMT_ONTIME_PAYMENT_divide_AMT_INSTALMENT_max==DEVIDE==AMT_INCOME_TOTAL
Wei_prev_AMT_DOWN_PAYMENT_divide_AMT_GOODS_PRICE_max==DEVIDE==AMT_INCOME_TOTAL
Wei_PREV_CHANNEL_TYPE_Credit and cash offices_MEAN==DEVIDE==AMT_INCOME_TOTAL
Wei_prev_AMT_ANNUITY_divide_DAYS_TERMINATION_max==DEVIDE==AMT_INCOME_TOTAL
Wei_creditcard_AMT_CREDIT_LIMIT_ACTUAL_divide_AMT_DRAWINGS_CURRENT_mean==DEVIDE==AMT_INCOME_TOTAL
Wei_prev_DAYS_FIRST_DRAWING_minus_DAYS_LAST_DUE_min==DEVI

 97%|█████████▋| 764/790 [00:06<00:00, 119.97it/s]

Wei_creditcard_AMT_BALANCE_divide_AMT_PAYMENT_TOTAL_CURRENT_median==DEVIDE==AMT_INCOME_TOTAL
Wei_prev_AMT_ANNUITY_max==DEVIDE==AMT_INCOME_TOTAL
Wei_APPROVED_AMT_APPLICATION_MEAN==DEVIDE==AMT_INCOME_TOTAL
Wei_prev_DAYS_FIRST_DUE_minus_DAYS_LAST_DUE_min==DEVIDE==AMT_INCOME_TOTAL
Wei_bureau_credit_enddate_percentage==DEVIDE==AMT_INCOME_TOTAL
Wei_prev_AMT_ANNUITY_divide_DAYS_LAST_DUE_max==DEVIDE==AMT_INCOME_TOTAL
Wei_prev_AMT_ANNUITY_divide_DAYS_FIRST_DRAWING_max==DEVIDE==AMT_INCOME_TOTAL
Wei_APPROVED_AMT_APPLICATION_MAX==DEVIDE==AMT_INCOME_TOTAL
Wei_prev_CNT_PAYMENT_sum==DEVIDE==AMT_INCOME_TOTAL
Wei_last_20by100_fraction_installment_paid_late_in_days_mean==DEVIDE==AMT_INCOME_TOTAL
Wei_inst_normalslope_AMT_LATE_PAYMENT_mean_sum==DEVIDE==AMT_INCOME_TOTAL
Wei_bureau_AMT_CREDIT_SUM_LIMIT_divide_DAYS_CREDIT_UPDATE_min==DEVIDE==AMT_INCOME_TOTAL
Wei_creditcard_AMT_PAYMENT_CURRENT_divide_CNT_DRAWINGS_ATM_CURRENT_median==DEVIDE==AMT_INCOME_TOTAL
Wei_prev_AMT_DOWN_PAYMENT_divide_DAYS_FIRST_DRAWING_

100%|██████████| 790/790 [00:06<00:00, 121.27it/s]

Wei_creditcard_CNT_DRAWINGS_CURRENT_sum==DEVIDE==AMT_INCOME_TOTAL
Wei_prev_AMT_APPLICATION_max==DEVIDE==AMT_INCOME_TOTAL
Wei_CC_AMT_DRAWINGS_ATM_CURRENT_VAR==DEVIDE==AMT_INCOME_TOTAL
Wei_bureau_AMT_CREDIT_SUM_OVERDUE_divide_DAYS_CREDIT_min==DEVIDE==AMT_INCOME_TOTAL
Wei_Wei_CNT_installment_per_version_STD==DEVIDE==AMT_INCOME_TOTAL
Wei_Wei_Remaining_AMT_Payment_TOTAL==DEVIDE==AMT_INCOME_TOTAL
Wei_Wei_CNT_installment_per_version_LAST==DEVIDE==AMT_INCOME_TOTAL
Wei_Wei_Remaining_AMT_Payment_LAST==DEVIDE==AMT_INCOME_TOTAL
Wei_Wei_Remaning_CNT_Payment_TOTAL==DEVIDE==AMT_INCOME_TOTAL
Wei_Wei_Remaing_Payment_Ratio_CURR==DEVIDE==AMT_INCOME_TOTAL
Wei_Wei_TOTAl_NEEDPAY_INCOME_RATIO==DEVIDE==AMT_INCOME_TOTAL
Wei_BU_CREDIT_SUM==DEVIDE==AMT_INCOME_TOTAL
Wei_BU_AMT_CREDIT_SUM_DEBT==DEVIDE==AMT_INCOME_TOTAL
Wei_BU_AMT_CREDIT_MAX_OVERDUE==DEVIDE==AMT_INCOME_TOTAL
Wei_APR_cashloan_riskscore_mean==DEVIDE==AMT_INCOME_TOTAL
Wei_Wei_EXT_SOURCE_1_x==DEVIDE==AMT_CREDIT==DEVIDE==AMT_INCOME_TOTAL
Wei_Wei_EXT_SOU

In [13]:
X = X.drop(['AMT_INCOME_TOTAL','AMT_GOODS_PRICE'],axis=1)
train = X.iloc[:307511].copy()
train['TARGET'] = target
print(train.shape)

test = X.iloc[307511:].copy()

(307511, 1165)


In [14]:
train.to_pickle(ENV.lgb_train_0828.value)
test.to_pickle(ENV.lgb_test_0828.value)

# Select Categorical columns

In [4]:
cates = []
tr = 10
for col in tqdm(X.columns):
    if X[col].nunique() <= tr:
        le = LabelEncoder()
       
        X[col] = X[col].fillna('NOTFOUND!')
        X[col] = X[col].astype(str)
        X[col] = le.fit_transform(X[col])
        X[col] = X[col].astype('category')
       
        if col.find('DOCUMENT') == -1 and col.find('WEEKDAY')== -1:
            cates.append(col)
            
    if X[col].dtypes == object:
        cates.append(col)
        X[col] = X[col].fillna('NOTFOUND!')
        X[col] = le.fit_transform(X[col])
        X[col] = X[col].astype('category')
print(len(cates))

100%|██████████| 802/802 [00:13<00:00, 61.24it/s] 

27


In [6]:
train = X.iloc[:307511].copy()
train['TARGET'] = target

test = X.iloc[307511:].copy()

In [6]:
seed=20
num_folds =5

def get_column_name(cols):
    return 'Wei_agg_mean_' + '==!=='.join(cols)


def get_groups(df,cols):
    dff = df[list(cols)].copy()
    name = 'Wei_Aggre_'
    seri = df[cols[0]].astype('str')
    for col in cols[1:]:
        name = name+'_'+col
        seri += '_'+dff[col].astype('str')
    return seri,name

folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)

for c in combinations(cates,2):
        col_name = get_column_name(c)
        train[col_name] = 0
        test[col_name] = 0

train_index = []
val_index = []
for tr_inx,te_inx in folds.split(train,train['TARGET']):
    train_index.append(tr_inx)
    val_index.append(te_inx)

        
print(train.shape)
print('start...')

for c in tqdm(combinations(cates,2)):
    for k in range(num_folds):
        col_name = get_column_name(c)
        
        seri, group_name = get_groups(train,c)
        train[group_name] = seri
        seri, group_name = get_groups(test,c)
        test[group_name] = seri
        mean_v = train.iloc[train_index[k]].groupby(group_name).TARGET.mean()
        train.loc[te_inx,col_name] = train.loc[val_index[k],group_name].map(mean_v)
        test[col_name] += 1/num_folds*test[group_name].map(mean_v)
    train = train.drop(group_name,axis=1)
    test = test.drop(group_name,axis=1)
    print(col_name)
    print(train.shape)
    print(test.shape)
    print('--------')
        
        

0it [00:00, ?it/s]

(307511, 1154)
start...


1it [00:08,  8.36s/it]

Wei_agg_mean_CODE_GENDER_x==!==REGION_RATING_CLIENT_x
(307511, 1154)
(48744, 1153)
--------


2it [00:16,  8.25s/it]

...

Wei_agg_mean_Bureau_balance_Last_STATUS_category==!==cnt_cash_30days_late
(307511, 1154)
(48744, 1153)
--------


350it [47:20,  8.12s/it]

Wei_agg_mean_Bureau_balance_Last_STATUS_category==!==cnt_cons_30days_late
(307511, 1154)
(48744, 1153)
--------


351it [47:28,  8.12s/it]

Wei_agg_mean_cnt_cash_30days_late==!==cnt_cons_30days_late
(307511, 1154)
(48744, 1153)
--------


In [9]:
# train764 = pd.read_pickle(ENV.lightgbm_train_764.value)
# test764 = pd.read_pickle(ENV.lightgbm_test_764.value)

# train['SK_ID_CURR'] = train764['SK_ID_CURR']
# print(train.shape)
# test['SK_ID_CURR'] = test764['SK_ID_CURR']
# print(test.shape)

In [12]:
drops = ['Wei_external_sources_min==DEVIDE==AMT_CREDIT']

In [13]:
train = train.drop(drops,axis=1)
test = test.drop(drops,axis=1)

In [16]:
train.to_pickle(ENV.lgb_train_0827_raw.value)
test.to_pickle(ENV.lgb_test_0827_raw.value)